In [ ]:
import mysql.connector
from mysql.connector import errorcode
import os
import re
import pandas as pd
import hvplot.pandas
from sqlalchemy import create_engine

# Connection
def connect():
    try:
        #REPLACE YOUR SQL INFORMATION WITH YOUR INFORMATION SUCH AS USER, PASSWORD, HOST, & DATABASE
        con = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')
        print('Connection successful')
        return con
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
            con.rollback()
            
            
#---------------------------------------------------------#
#---------------------------------------------------------#            


#Insert Record
def add(con):
    try:
        #PROMPTING INPUTS OF THE RESORT'S INFORMATION
        while True:
            Resort = input('Enter your Full Resort Name: ')
            Owner = input('Please enter your full name(Owner): ')
            Address = input("Please enter the exact address of the resort: ")
            while True:
                Email = input('Please enter your email address: ')
                confirm_email = re.search(".+@gmail|hotmail|outlook|yahoo|\.com", Email)
                if confirm_email:
                    break
                else:
                    print ("Invalid email address.\n Please enter a valid email.")
            while True:
                Contact = input('Please enter your Contact Number: ')
                confirm_contact = re.match(r'^\d{7}$',Contact)
                if confirm_contact:
                    break
                else:
                    print ("Invalid contact number.\n Please enter a valid contact number.")
            cursor = con.cursor()
            
            #HERE, WILL FILL THE RESORT'S INFORMATION COLUMNS FROM THE INPUTS ABOVE
            #Inserting into RESORT INFO
            sql = "Insert Into Resort_Information (Resort_Name,Owner_Name,Address,Contact_Number,Email_Address) Values ('%s', '%s', '%s','%s','%s')" % (Resort, Owner, Address, Contact, Email)
            cursor.execute(sql)
            con.commit()
            
            #PROMPTING INPUTS OF THE RESORT'S DESCRIPTION
            ResortName = Resort
            OwnerName = Owner
            Capacity = int(input("Enter maximum capacity of the resort: "))
            RoomQty = int(input("Enter number of rooms: "))
            NipaHutQty = int(input("Enter number of nipa hut: "))
            Tent = input("Tent Camping availability [Yes/No]: ")
            
            #HERE, WILL FILL THE RESORT'S DESCRIPTION COLUMNS FROM THE RESORT'S DESC. INPUTS
            #Inserting into RESORT DESC
            sql1 = "Insert Into Resort_Description (Resort_Name,Owner_Name,Capacity,Room_Quantity,Nipa_Hut_Quantity,Tent_Camping) Values ('%s','%s','%d', '%d', '%d','%s')" % (ResortName,OwnerName,Capacity, RoomQty,NipaHutQty,Tent)
            cursor.execute(sql1)
            con.commit()
            
            #HERE, WILL FILL THE OWNER'S DESCRIPTION COLUMNS FROM THE RESORT'S DESC. INPUTS
            #Inserting into OWNER INFO
            sql1 = "Insert Into Owner_Information (Full_Name,Email_Address,Contact_Number) Values ('%s','%s','%s')" % (Owner,Email,Contact)
            cursor.execute(sql1)
            con.commit()
            
            print('New Resort added and save!')
            x = input("Do you want to add another Resort? [y/n]? ")
            if x == 'y':
                os.system('cls')
                continue
            else:
                break

    except mysql.connector.Error as err:
            print(err)
            con.rollback()
    else:
        cursor.close()
        con.close()
        
        
#---------------------------------------------------------#
#---------------------------------------------------------#
        

#Delete Record
def delete (con):
    try:
        while True:
            Resort = input('Enter your Full Resort Name: ')
            Owner = input("Enter the Owner of the Resort: ")
            
            #Deleting OWNER INFO
            cursor = con.cursor()
            sql2 = "Delete From Owner_Information Where Full_Name = '%s'" % Owner
            cursor.execute(sql2)
            con.commit()

            #Deleting RESORT DESC
            cursor = con.cursor()
            sql3 = "Delete From Resort_Description Where Resort_Name = '%s'" % Resort
            cursor.execute(sql3)
            con.commit()
            
            #Deleting RESORT INFO
            cursor = con.cursor()
            sql1 = "Delete From Resort_Information Where Resort_Name = '%s'" % Resort
            cursor.execute(sql1)
            con.commit()
            
            print('Resort deleted')
            x = input("Do you want to delete another resort? [y/n]? ")
            if x == 'y':
                os.system('cls')
                continue
            else:
                break

    except mysql.connector.Error as err:
        print(err)
        con.rollback()
    else:
        cursor.close()
        con.close()

        
#---------------------------------------------------------#
#---------------------------------------------------------#
        
        
#Edit Record
def edit(con):
    try:
        while True:
            Resort = input('Enter your Full Resort Name: ')
            Owner = input('Please enter your full name(Owner): ')
            
            ResortName = Resort
            OwnerName = Owner
            Capacity = int(input("Enter NEW maximum capacity of the resort: "))
            RoomQty = int(input("Enter NEW number of rooms: "))
            NipaHutQty = int(input("Enter NEW number of nipa hut: "))
            Tent = input("NEW Tent Camping availability [Yes/No]: ")


            # Execute SQL
            cursor = con.cursor()
            sql = "Update Resort_Description Set Owner_Name = '%s', Capacity = '%d', Room_Quantity = '%d', Nipa_Hut_Quantity = '%d', Tent_Camping = '%s' Where Resort_Name = '%s'" % (Owner, Capacity,RoomQty,NipaHutQty,Tent,Resort)
            cursor.execute(sql)
            con.commit()

            print('ResortChanges save')
            x = input("Do you want to edit another resort? [y/n]? ")
            if x == 'y':
                os.system('cls')
                continue
            else:
                break

    except mysql.connector.Error as err:
            print(err)
            con.rollback()
    else:
        cursor.close()
        con.close()

        
#---------------------------------------------------------#
#---------------------------------------------------------#
        
        
#Search Record
def search(con):
    try:
        while True:
            Resort = input('Enter your Full Resort Name: ')

            # Executing SQL-DML Commands
            cursor = con.cursor()
            query = "Select * From Resort_Description Where Resort_Name ='%s'" % Resort

            # Search Product
            cursor.execute(query)
            row = cursor.fetchone()
            if row is not None:
                print ("Resort Name:",Resort)
                print('Owner Name :', row[1])
                print('Capacity:', row[2])
                print ("Room Quantity:",row[3])
                print ("Nipa_Hut_Quantity:",row[4])
                print ("Tent Camping Availability:",row[5])
            else:
                print("Resort not found")

            x = input("Do you want to search another resort? [y/n]? ")
            if x == 'y':
                os.system('cls')
                continue
            else:
                break

    except mysql.connector.Error as err:
            print(err)
            con.rollback()
    else:
        cursor.close()
        con.close()

        
#---------------------------------------------------------#
#---------------------------------------------------------#
        
        
#Retrieve all records
def searchAll(con):
    # Executing SQL-DML Commands
    cursor = con.cursor()
    query = "SELECT * FROM Resort_Description"
    cursor.execute(query)
    for rows in cursor:
        print (rows)

#---------------------------------------------------------#
#---------------------------------------------------------#
    

def main():
    print ("Welcome to Alibijaban Island!")
    print ("Explore Resorts as:")
    print ("[T] - Tourist")
    print ("[A] - Admin")
    role = input("Enter role [T/A]: ")
    if role == "A":
        #READS THE FILE
        file = open("D:/Jupyter Ali Resorts/admin_acc_distribution.txt", 'r')
        contents = file.read()
    
        #ASSIGNED EMPTY VALUES FOR EMAIL AND PASSWORD
        email_confirmed_2 = ""
        pass_confirmed_2 = ""
    
        #WILL LOOP LINE OF CODES IN VARIOUS CONDITIONS
        A = True
        while A == True:

            #WILL LOOP LINE OF CODES IN VARIOUS CONDITIONS
            B = True
            while B == True:
                email_2 = input("Enter your email address: ")

                #ASK INPUT OF YOUR EMAIL ADDRESS
                confirm_email_2 = re.search(".+@gmail|hotmail|outlook|yahoo|\.com", email_2)

                #CONDITIONS
                if confirm_email_2:

                    #INPUTTED EMAIL WILL BE ASSIGNED TO THE ASSIGNED EMPTY CONFIRMED EMAIL VALUE
                    email_confirmed_2 = email_2
                    break
                else:

                    #IF EMAIL IS INVALID
                    time.sleep(2)
                    print ("Invalid email address!")
                    print ("Please try again.")

            pass_2 = input("Please enter your password: ")

            #INPUTTED PASSWORD WILL BE ASSIGNED TO THE ASSIGNED EMPTY CONFIRMED PASSWROD VALUE
            pass_confirmed_2 = pass_2

            d = email_confirmed_2+':'+pass_confirmed_2

            #CONDITIONS

            #IF EMAIL AND PASSWORD ALREADY IN THE DESIGNATED ACCOUNT DISTRIBUTION FILE
            if d in contents:
                print ("Select below what to do.")
                C = True
                while C == True:
                    print("[1] Add New Resort")
                    print("[2] Edit Resort")
                    print("[3] Delete Resorts")
                    print("[4] Search Resorts")
                    print("[5] Display All Resorts")
                    choice = input("Enter your choice : ")

                    if choice == "1":
                        os.system('cls')
                        c = connect()
                        add(c)
                    elif choice == "2":
                        os.system('cls')
                        c = connect()
                        edit(c)
                    elif choice == "3":
                        os.system('cls')
                        c = connect()
                        delete(c)
                    elif choice == "3":
                        os.system('cls')
                        c = connect()
                        delete(c)
                    elif choice == "4":
                        os.system('cls')
                        c = connect()
                        search(c)
                    elif choice == "5":
                        os.system('cls')
                        c = connect()
                        searchAll(c)

                    x = input("Return to main menu [y/n]? ")
                    if x == 'y':
                        os.system('cls')
                        continue
                    else:
                        C = False
                        B = False
                        A = False
                        
                    exit(0)
            else:
                #IF EMAIL AND PASSWORD IS NOT YET IN THE DESIGNATED ACCOUNT DISTRIBUTION FILE
                print ("Account not found.")
                x = input("Want to try again?:  [y/n]? ")
                if x == 'y':
                    os.system('cls')
                    continue
                else:
                    break

                exit(0)
        
            
            
    elif role == "T":
        print ("Select below what to do.")
        D = True
        while D == True:
            print("[1] Add New Resort")
            print("[2] Edit Resort")
            print("[3] Delete Resorts")
            print("[4] Search Resorts")
            print("[5] Display All Resorts")
            choice = input("Enter your choice : ")

            if choice == "1":
                os.system('cls')
                print ("You cannot perform this action.\n Only admins are permitted.")
            elif choice == "2":
                os.system('cls')
                print ("You cannot perform this action.\n Only admins are permitted.")
            elif choice == "3":
                os.system('cls')
                print ("You cannot perform this action.\n Only admins are permitted.")
            elif choice == "3":
                os.system('cls')
                c = connect()
                delete(c)
            elif choice == "4":
                os.system('cls')
                c = connect()
                search(c)
            elif choice == "5":
                os.system('cls')
                c = connect()
                searchAll(c)
                
            x = input("Return to main menu [y/n]? ")
            if x == 'y':
                os.system('cls')
                continue
            else:
                break

            exit(0)


main()

In [ ]:
#FOR OPTION 5 (Retrieve All Resorts Description)
import pandas as pd
import hvplot.pandas
import mysql.connector

cnx = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')

df = pd.read_sql('SELECT * FROM Resort_Description', cnx)
df.hvplot.table(height=200,width=500)

In [ ]:
#FOR OPTION 5 (Retrieve All Resorts Information)
import pandas as pd
import hvplot.pandas
import mysql.connector

cnx = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')

df = pd.read_sql('SELECT * FROM Resort_Information', cnx)
df.hvplot.table(height=300,width=850)

In [ ]:
#FOR OPTION 5 (Retrieve All Owner Information)
import pandas as pd
import hvplot.pandas
import mysql.connector

cnx = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')

df = pd.read_sql('SELECT * FROM Owner_Information', cnx)
df.hvplot.table(height=200,width=500)

In [ ]:
#FOR OPTION 4 (Search for a specific Resort)
import pandas as pd
import hvplot.pandas
import mysql.connector

cnx = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')

Resort = input('Enter your Full Resort Name: ')
df = pd.read_sql("Select * From Resort_Description Where Resort_Name ='%s'" % Resort,cnx)
df.hvplot.table(height=200,width=900)

In [ ]:
import pandas as pd
import hvplot.pandas
import mysql.connector

cnx = mysql.connector.connect(user='root', password='Aco11161982', host='localhost', database='AliIslandResortsDB')

df = pd.read_sql('SELECT * FROM Resort_Description', cnx)
df = df.groupby('Resort_Name')[['Capacity']].sum()
df.hvplot(kind='bar',color="Red")
